In [1]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import seaborn as sns
import json

In [2]:
json_champ = pd.DataFrame()

In [3]:
data = pd.read_csv('11.23.csv', index_col=0)

C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [4]:
value = data['championId'].value_counts()

In [5]:
champId = value.index

In [6]:
champId = champId.sort_values( ascending=False) 

In [7]:
champId

Int64Index([887, 876, 875, 777, 711, 555, 526, 523, 518, 517,
            ...
             10,   9,   8,   7,   6,   5,   4,   3,   2,   1],
           dtype='int64', length=157)

In [8]:
list = champId
json_id = []
for i in list:
    file = open(f'{i}.json', 'rt', encoding='UTF8')
    globals()[f'champ_{i}'] = json.load(file)
    json_id.append(globals()[f'champ_{i}'])

In [9]:
print(champ_887.get('roles'))

['fighter', 'assassin']


In [10]:
style = []
difficulty = []
damageType = []
damage = []
durability = []
crowdControl = []
mobility = []
utility = []
championId = []
name = []
roles = []

In [11]:
list = json_id
for num in list:
    style.append(num.get("tacticalInfo").get('style'))
    difficulty.append(num.get("tacticalInfo").get('difficulty'))
    damageType.append(num.get("tacticalInfo").get('damageType'))
    damage.append(num.get("playstyleInfo").get('damage'))
    durability.append(num.get("playstyleInfo").get('durability'))
    crowdControl.append(num.get("playstyleInfo").get('crowdControl'))
    mobility.append(num.get("playstyleInfo").get('mobility'))
    utility.append(num.get("playstyleInfo").get('utility'))
    championId.append(num.get("id"))
    name.append(num.get("name"))
    a = []
    for list in num.get("roles"):
        a.append(list)
    roles.append(a)

In [12]:
json_champ['championId'] = championId
json_champ['name'] = name
json_champ['style'] = style
json_champ['difficulty'] = difficulty
json_champ['damageType'] = damageType
json_champ['damage'] = damage
json_champ['durability'] = durability
json_champ['crowdControl'] = crowdControl
json_champ['mobility'] = mobility
json_champ['utility'] = utility
json_champ['roles'] = roles

In [13]:
json_champ

,championId,name,style,difficulty,damageType,damage,durability,crowdControl,mobility,utility,roles
0,887,Gwen,5,2,kMagic,3,2,1,3,1,"[fighter, assassin]"
1,876,Lillia,8,3,kMagic,3,1,2,2,2,"[fighter, mage]"
2,875,Sett,4,2,kPhysical,2,2,2,2,1,"[fighter, tank]"
3,777,Yone,5,3,kMixed,3,1,2,3,1,"[assassin, fighter]"
4,711,Vex,9,2,kMagic,3,2,2,2,1,[mage]
...,...,...,...,...,...,...,...,...,...,...,...
152,5,Xin Zhao,4,1,kPhysical,2,2,2,2,1,"[fighter, assassin]"
153,4,Twisted Fate,7,2,kMagic,3,1,2,2,2,[mage]
154,3,Galio,10,2,kMagic,2,3,3,2,1,"[tank, mage]"
155,2,Olaf,4,2,kPhysical,2,2,2,1,1,"[fighter, tank]"


In [14]:
json_champ.to_csv('json_champ.csv',index=False, encoding="utf-8-sig")

In [15]:
json_champ['style'].value_counts()

10    32
8     19
7     18
9     18
3     17
4     15
6     11
2     10
5     10
1      7
Name: style, dtype: int64

In [16]:
json_champ['damageType'].value_counts()

kMagic       78
kPhysical    63
kMixed       16
Name: damageType, dtype: int64

In [17]:
json_champ['damageType']

0         kMagic
1         kMagic
2      kPhysical
3         kMixed
4         kMagic
         ...    
152    kPhysical
153       kMagic
154       kMagic
155    kPhysical
156       kMagic
Name: damageType, Length: 157, dtype: object

In [18]:
constant_patch = '11.23.1'

# 라이엇 개발자 페이지에서 최신 버전 한국 서버 챔피언 정보 가져오기
champion_info = requests.get("http://ddragon.leagueoflegends.com/cdn/" + constant_patch + "/data/ko_KR/champion.json").json()

In [19]:
# 챔피언 정보 => DataFrame으로 변환
# champion_df는 'key'로 id값, 'name'으로 챔피언 한국어 이름을 가지는 156rows짜리 DataFrame
champion_df = pd.DataFrame(champion_info['data']).T[['key', 'name']]

# champion_df의 'key' column string => numeric 변환
champion_df['key'] = pd.to_numeric(champion_df['key'])

In [20]:
champion_df

,key,name
Aatrox,266,아트록스
Ahri,103,아리
Akali,84,아칼리
Akshan,166,아크샨
Alistar,12,알리스타
...,...,...
Zed,238,제드
Ziggs,115,직스
Zilean,26,질리언
Zoe,142,조이


In [21]:
json_champ = json_champ.merge(champion_df, how = 'left', left_on = 'championId', right_on = 'key')

In [22]:
json_champ

,championId,name_x,style,difficulty,damageType,damage,durability,crowdControl,mobility,utility,roles,key,name_y
0,887,Gwen,5,2,kMagic,3,2,1,3,1,"[fighter, assassin]",887,그웬
1,876,Lillia,8,3,kMagic,3,1,2,2,2,"[fighter, mage]",876,릴리아
2,875,Sett,4,2,kPhysical,2,2,2,2,1,"[fighter, tank]",875,세트
3,777,Yone,5,3,kMixed,3,1,2,3,1,"[assassin, fighter]",777,요네
4,711,Vex,9,2,kMagic,3,2,2,2,1,[mage],711,벡스
...,...,...,...,...,...,...,...,...,...,...,...,...,...
152,5,Xin Zhao,4,1,kPhysical,2,2,2,2,1,"[fighter, assassin]",5,신 짜오
153,4,Twisted Fate,7,2,kMagic,3,1,2,2,2,[mage],4,트위스티드 페이트
154,3,Galio,10,2,kMagic,2,3,3,2,1,"[tank, mage]",3,갈리오
155,2,Olaf,4,2,kPhysical,2,2,2,1,1,"[fighter, tank]",2,올라프
